In [120]:
import ast
import json

import pandas as pd
import numpy as np

from zoobot import label_metadata
from zoobot.estimators import losses

In [121]:
m0 = pd.read_csv('/home/walml/repos/zoobot/temp/gz2_allq_it0_m0.csv')
m1 = pd.read_csv('/home/walml/repos/zoobot/temp/gz2_allq_it0_m1.csv')



In [122]:
assert np.all(m0['dr7objid'] == m1['dr7objid'])

In [123]:

questions = label_metadata.gz2_questions
# gz2 cols
version = 'gz2'
label_cols = label_metadata.gz2_label_cols

schema = losses.Schema(label_cols, questions, version=version)

{smooth-or-featured, indices 0 to 1, asked after None: (0, 1), disk-edge-on, indices 2 to 3, asked after <zoobot.estimators.losses.Answer object at 0x7fe57f7551d0>: (2, 3), has-spiral-arms, indices 4 to 5, asked after <zoobot.estimators.losses.Answer object at 0x7fe57f755c90>: (4, 5), bar, indices 6 to 7, asked after <zoobot.estimators.losses.Answer object at 0x7fe57f755c90>: (6, 7), bulge-size, indices 8 to 11, asked after <zoobot.estimators.losses.Answer object at 0x7fe57f755c90>: (8, 11), something-odd, indices 12 to 13, asked after None: (12, 13), how-rounded, indices 14 to 16, asked after <zoobot.estimators.losses.Answer object at 0x7fe57f7557d0>: (14, 16), bulge-shape, indices 17 to 19, asked after <zoobot.estimators.losses.Answer object at 0x7fe57f755590>: (17, 19), spiral-winding, indices 20 to 22, asked after <zoobot.estimators.losses.Answer object at 0x7fe57f755b90>: (20, 22), spiral-count, indices 23 to 28, asked after <zoobot.estimators.losses.Answer object at 0x7fe57f755b9

In [124]:
m_all = m0.copy()
for question in schema.questions:
    for answer in question.answers:
        m0_conc = m0[answer.text + '_concentration'].apply(ast.literal_eval)
        m1_conc = m1[answer.text + '_concentration'].apply(ast.literal_eval)
        
#         print(m0_conc)
        concentrations = [(a, b) for a, b in zip(m0_conc, m1_conc)]
#         concentrations = np.stack([m0_conc, m1_conc], axis=-1)
        m_all[answer.text + '_concentration'] = [json.dumps(x) for x in concentrations]
#         print(list(zip(m0_conc, m1_conc))[0])
        mean_concentrations = np.mean(concentrations, axis=1).mean(axis=1)  # model and dropout
#         print(mean_concentrations.shape)
        m_all[answer.text + '_concentration_mean'] = mean_concentrations
#         break
#     break
                                                         
                                                         #         print(concentrations[0])
#         

In [125]:
# np.array(concentrations).shape

In [126]:
m_all[answer.text + '_concentration'].iloc[0]

'[[34.94941329956055, 39.27217102050781, 35.92841339111328, 39.33571243286133, 38.794002532958984], [34.602569580078125, 39.943756103515625, 38.082122802734375, 38.76411819458008, 40.76685333251953]]'

In [127]:
# np.array(m_all[answer.text + '_concentration'].iloc[0]).shape

In [128]:
m_all[answer.text + '_concentration_mean']

0        38.043913
1         7.973587
2        27.516973
3        31.797214
4         1.305081
           ...    
14995    20.576980
14996    42.553918
14997    49.952033
14998    25.015487
14999     4.502175
Name: spiral-count_cant-tell_concentration_mean, Length: 15000, dtype: float64

In [129]:
subject_df = pd.read_csv('/media/walml/beta/galaxy_zoo/gz2/subjects/gz2_classifications_and_subjects.csv', usecols=['dr7objid', 'location'])

In [130]:
subject_df['dr7objid'].value_counts()

587735697522557119    1
588023239669842054    1
587731499189272666    1
587741816248467606    1
587729157908136098    1
                     ..
588017724397256860    1
587735343191621644    1
587736977420255436    1
587736477049094284    1
588017992302526501    1
Name: dr7objid, Length: 239695, dtype: int64

In [131]:
for_app = pd.merge(m_all, subject_df, on='dr7objid', how='inner')

In [132]:
for_app[answer.text + '_concentration_mean']

0        38.043913
1         7.973587
2        27.516973
3        31.797214
4         1.305081
           ...    
14995    20.576980
14996    42.553918
14997    49.952033
14998    25.015487
14999     4.502175
Name: spiral-count_cant-tell_concentration_mean, Length: 15000, dtype: float64

In [133]:
for_app['location']

0         http://s3.amazonaws.com/zoo2/27919.jpg
1        http://s3.amazonaws.com/zoo2/227990.jpg
2        http://s3.amazonaws.com/zoo2/119556.jpg
3         http://s3.amazonaws.com/zoo2/20974.jpg
4        http://s3.amazonaws.com/zoo2/208468.jpg
                          ...                   
14995     http://s3.amazonaws.com/zoo2/62151.jpg
14996     http://s3.amazonaws.com/zoo2/86103.jpg
14997    http://s3.amazonaws.com/zoo2/170029.jpg
14998     http://s3.amazonaws.com/zoo2/40130.jpg
14999    http://s3.amazonaws.com/zoo2/158659.jpg
Name: location, Length: 15000, dtype: object

In [134]:
for_app.to_csv('/home/walml/repos/galaxy-poster/gz2_classifications.csv', index=False)